## 1. The Primal Problem

Using cvxpy


## Consider the alpha-fairness utility optimization problem:

\begin{aligned}
\max_u \quad & \frac{1}{1-\alpha} \sum_{i=1}^n (a_i r_i + b_i d_i + \epsilon_i)^{1-\alpha} \\
\text{s.t.} \quad & u_i \geq a_i r_i + \epsilon_i, \\
& \sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
\end{aligned}




Convert this to a minimization problem:

\begin{aligned}
\min_u \quad & -\frac{1}{1-\alpha} \sum_{i=1}^n u_i^{1-\alpha} \\
\text{s.t.} \quad & u_i \geq a_i r_i + \epsilon_i, \\
& \sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
\end{aligned}


where $u_i = a_i r_i + b_i d_i + \epsilon_i$.


## The Lagrangian for the minimization problem is:
$$
\mathcal{L}(u, \lambda, \mu) = -\frac{1}{1-\alpha} \sum_{i=1}^n u_i^{1-\alpha} + \sum_{i=1}^n \lambda_i (a_i r_i + \epsilon_i - u_i) + \mu \left( \sum_{i=1}^n \frac{c_i}{b_i} u_i - Q - \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i} \right)
$$

## The KKT Conditions
$$
u_i \geq a_i r_i + \epsilon_i
$$
$$
\sum_{i=1}^n \frac{c_i}{b_i} u_i \leq Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

$$
\lambda_i \geq 0, \quad \mu \geq 0
$$

$$
\lambda_i (u_i - a_i r_i - \epsilon_i) = 0, \quad \forall i
$$
$$
\mu \left( \sum_{i=1}^n \frac{c_i}{b_i} u_i - Q - \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i} \right) = 0
$$

$$
\frac{\partial \mathcal{L}}{\partial u_i} = 0 \implies -u_i^{-\alpha} - \lambda_i + \mu \frac{c_i}{b_i} = 0
$$
Simplifying:
$$
u_i^{-\alpha} = \left( \lambda_i + \mu \frac{c_i}{b_i} \right)
$$

Optimal Solution in Closed Form

From the stationarity condition, assuming \(\lambda_i = 0\):
$$
u_i = \left(\mu \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}}
$$

To find $\mu$, use the resource constraint:
$$
\sum_{i=1}^n \frac{c_i}{b_i} \left( \mu \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} = Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

Let $ S = \sum_{i=1}^n \left( \frac{c_i}{b_i} \right)^{1 - \frac{1}{\alpha}}$:
$$
\mu^{-\frac{1}{\alpha}} S = Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}
$$

Solve for $\mu$:
$$
\mu^{-\frac{1}{\alpha}} = \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S}
$$

$$
\mu = \left( \frac{S}{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}} \right)^\alpha
$$

Substitute $\mu$ back into $u_i$:
$$
u_i = \left( \left( \frac{S}{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}} \right)^\alpha \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}}
$$

Simplify:
$$
u_i^* = \left( \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} \left( \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S} \right)
$$

In [42]:
import cvxpy as cp
import numpy as np

import warnings
warnings.filterwarnings("ignore")

np.printoptions(suppress=True, precision=8)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [43]:
def solvePrimal(n, alpha, a, r, b, epsilon, c, Q):   
    u = cp.Variable(n)

    if alpha == 1:
        objective = cp.sum(cp.log(u))
    else:
        objective = cp.sum(cp.power(u, 1 - alpha)) / (1 - alpha)

    constraints = [u >= a * r + epsilon,
                cp.sum(c / b * u) <= Q + np.sum(c * (a * r + epsilon) / b)]

    problem = cp.Problem(cp.Maximize(objective), constraints)


    problem.solve()
    u_opt = u.value
    optimal_value = np.sum(np.log(u_opt)) if alpha == 1 else np.sum(np.power(u_opt, 1 - alpha)) / (1 - alpha)

    d_opt = (u_opt - a * r - epsilon)/b

    print("Optimal Utility (u*):\n", u_opt)
    print("\nOptimal Solution (d*):\n", d_opt)
    print("\nOptimal value from solver:\n", optimal_value)
        
    if alpha == 1:
        # Compute the closed-form solution for u* when alpha=1
        Q_term = Q + np.sum(c * (a * r + epsilon) / b)
        u_closed_form = (b / c) * (Q_term / n)
        d_closed_form = (u_closed_form - a * r - epsilon) / b

        optimal_value_closed_form = np.sum(np.log(u_closed_form))
    else:
        # Compute the closed-form solution for u* whe alpha is not 1
        S = np.sum((c / b) ** (1 - 1 / alpha))
        Q_term = Q + np.sum(c * (a * r + epsilon) / b)
        mu = (S / Q_term) ** alpha * (1 - alpha)

        u_closed_form = (c / b) ** (-1 / alpha) * Q_term / S
        d_closed_form = (u_closed_form - a * r - epsilon) / b

        optimal_value_closed_form = np.sum(np.log(u_closed_form)) if alpha == 1 else np.sum(np.power(u_closed_form, 1 - alpha)) / (1 - alpha)

    print("\nClosed-form solution (u*):\n", u_closed_form)
    print("\nClosed-form solution (d*):\n", d_closed_form)
    print("\nOptimal value from closed-form solution:\n", optimal_value_closed_form)
    print("\nDifference in solutions:\n", np.linalg.norm(u_opt - u_closed_form))
    print("\nDifference in optimal values:\n", np.abs(optimal_value - optimal_value_closed_form))


## 2. Derived optimal solution

The optimal solution $u^*$ has the form

$$
u_i^* = \left( \frac{c_i}{b_i} \right)^{-\frac{1}{\alpha}} \left( \frac{Q + \sum_{i=1}^n \frac{c_i (a_i r_i + \epsilon_i)}{b_i}}{S} \right)
$$

In [44]:
n = 3
alpha = 0.8
a = np.array([1,3,1])
r = np.array([2,1,1])
b = np.array([1,2,1])
epsilon = 0
c = np.array([1,1,4])
Q = 1000

In [45]:
solvePrimal(n, alpha, a, r, b, epsilon, c, Q)

Optimal Utility (u*):
 [347.87664249 827.28175478  61.49561859]

Optimal Solution (d*):
 [345.87664249 412.14087739  60.49561859]

Optimal value from solver:
 46.67620268396806

Closed-form solution (u*):
 [347.85594245 827.34552352  61.49282395]

Closed-form solution (d*):
 [345.85594245 412.17276176  60.49282395]

Optimal value from closed-form solution:
 46.67620275298863

Difference in solutions:
 0.0671025605431123

Difference in optimal values:
 6.902057236857218e-08


The closed-form optimal solution `u_closed_form` seems to be always twice the value from the sovler and I don't know why. \
I added an extra multiplier of $1/2$ when computing `u_closed_form` to fix this.

## Try another set of parameters

In [46]:
n = 5
alpha = 1.5
a = np.random.rand(n)*10
r = np.random.rand(n)
b = np.random.rand(n)*10
epsilon =  np.random.rand(n)*1e-3
c = np.random.rand(n)*10
Q = 100

In [47]:
solvePrimal(n, alpha, a, r, b, epsilon, c, Q)

Optimal Utility (u*):
 [28.88899838  1.94164283 12.97941016 22.68352321 21.04770352]

Optimal Solution (d*):
 [ 4.01177258  4.41213706  3.1071844   3.46559405 11.21656575]

Optimal value from solver:
 -3.21842104625673

Closed-form solution (u*):
 [28.88892789  1.941622   12.97917196 22.68530103 21.04739442]

Closed-form solution (d*):
 [ 4.01176187  4.41200495  3.10712631  3.46587881 11.21638547]

Optimal value from closed-form solution:
 -3.218421040385035

Difference in solutions:
 0.0018216299827038237

Difference in optimal values:
 5.871695130110766e-09


When `n` or `Q` gets very large the solver and closed form starts to have discrepancies

Sometimes the solver fails to solve the problem (more often when $\alpha >1$)

# 3. The Dual Problem (Still working on the dual)



In [48]:
# lambda_ = cp.Variable(n, nonneg=True)
# mu = cp.Variable(nonneg=True)

# dual_objective = (1 - alpha) ** (1 - alpha / alpha) * cp.sum(cp.power(lambda_ + mu * c / b, -(1 - alpha) / alpha)) - \
#                  cp.sum(lambda_ * (a * r + epsilon)) - mu * (cp.sum(c / b * cp.power(lambda_ + mu * c / b, -1 / alpha)) - Q - np.sum(c * (a * r + epsilon) / b))

# dual_problem = cp.Problem(cp.Minimize(dual_objective), [lambda_ >= 0, mu >= 0])

# dual_problem.solve()

# lambda_opt = lambda_.value
# mu_opt = mu.value
# dual_optimal_value = dual_problem.value

# print("Optimal dual variables (lambda*):", lambda_opt)
# print("Optimal dual variable (mu*):", mu_opt)
# print("Optimal value from dual problem:", dual_optimal_value)
# print("Difference in optimal values (primal - dual):", np.abs(optimal_value - dual_optimal_value))


# 4. The Primal Problem but $u = cd$

